# Data Science final year project : UEFA Champions League 2018-2019

<i>Cyril TSO and Tsiory RAZAFINDRAZAKA - Master 2 IoT & Big Data</i>

The goal of the project is to predict the winner of the UEFA Champions League 2018-2019 season.

In this notebook, only a part of the cleaning, the EDA and the Machine Learning part are gonna be displayed.

If you are looking for the Web Scraping and the Data Cleaning code, you can found them on my Github at this link : https://github.com/cyriltso/PFE-Data-Science

So, after having acquired the raw data from <i>ESPN, Whoscored and Footystats</i> and transformed them into a global dataset with Pandas, we're ready to analyse it and to find which Machine Learning is best suited for making the most accurate prediction regarding the winner of the UEFA Champions League.

##  Table of Contents

1. Exploratory Data Analysis <br>
2. Imputing & Converting the categorical data to numerical data
3. Making the predictions
4. Fine-Tuning the model (starting the notebook cycle again) <br>
5. Feature Engineering (reset of the cycle) <br>
6. Storytelling & Conclusion <br>

In [3]:
### Importing libraries needed for the whole process

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, roc_curve, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Imputer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier

### Content of the dataset

The dataset is composed of 210 observations (from 2011 to 2018) and 42 features.

Overall, the dataframe can be distinguished in 4 global categories concatened together :
    - The teams' stats in each round of the competition (starting from the Round of 16, so only the 16 best teams does appear in this section)
    - The teams' detailed stats in the whole competition (all of the teams that have played in the competition appear there)
    - The best scorers' stats
    - The best assists players' stats

### Dataset's features glossary

    - YEAR : year of each season
    - ROUND : round of the competition (round of 16, quarter-finals, semi-finals, final)
    - RANK ER : rank of each team in each round of the competition
    - TEAM ER : name of each team in each round of the competition
    - MP : number of match played by each team in each round of the competition (2 per round and 1 in the final)
    - W : number of wins by each team in each round of the competition
    - D : number of draws by each team in each round of the competition
    - L : number of losses by each team in each round of the competition
    - GF : number of goals scored by each team in each round of the competition
    - GD : goals difference from each team in each round of the competition
    - Last 5 : last results
    - CS% : clean sheet percentage (number of matches with goals conceded
    - BTTS% : percentage of matches where both teams scored a goal
    - FTS% : percentage of matches where a team failed to score a goal
    - Over 1.5+ % : percentage of matches where a team scores more than 1.5 goals
    - Over 2.5+ % : percentage of matches where a team scores more than 2.5 goals
    - AVG : goal average of a team
    - CATEGORIES 1 : first category of the dataset (detailed stats here)
    - RANK DS : team rank with the detailed stats
    - TEAM DS : team name with the detailed stats
    - GOALS : number of goals scored in total during the whole competition 
    - SHOTS : shots attempts average
    - YELLOW CARD : number of yellow cards
    - RED CARD : number of red cards
    - POSSESSION : possession percentage
    - PASS% : pass accuracy percentage
    - RATING : team evaluation (created by the forecaster)
    - CATEGORIES 2 : second category of the dataset (scorers here)
    - RANK SC : scorers ranking
    - NAME SC : scorers names
    - TEAM SC : scorers teams
    - MP SC : matches played by the scorers
    - G : number of goals scored by the scorers
    - GOALS RATIO : goals ratio
    - CATEGORIES 3 : third category of the dataset (assists players here)
    - RANK A : assists players ranking
    - NAME A : assists players names
    - TEAM A : assists players teams
    - MP A : matches played by the assists players
    - A : number of assists
    - ASSISTS RATiO : assists ratio

### Exploratory Data Analysis

The goal of the EDA is to make extra cleaning, analyse the data numerically and graphically in order to :

    - Have an overview if the dataset (in order to make adjustments)
    - Make extra cleaning (if necessary)
    - Splitting the dataset into subsets
    - Making descriptive and graphical analysis
    - Find biases (abnormal values) in the features
    - Determine correlation between features

#### Overviewing the data

In [4]:
### Loading the dataset

file = 'UCL Dataset.csv'

df = pd.read_csv(file)

In [5]:
### Overview of the dataset

df.shape

(210, 42)

In [6]:
df.head()

,YEAR,ROUND,RANK ER,TEAM ER,MP,W,D,L,GF,GA,...,MP SC,G,GOALS RATIO,CATEGORIES 3,RANK A,NAME A,TEAM A,MP A,A,ASSISTS RATIO
0,2011/12,8th Finals,1,FC Barcelona,2,2,0,0,10,2,...,11,14,1.27,ASSISTS,1,Lionel Messi,FC Barcelona,11,9,0.82
1,2011/12,8th Finals,2,Real Madrid,2,1,1,0,5,2,...,11,12,1.09,ASSISTS,2,Karim Benzema,Real Madrid,10,5,0.50
2,2011/12,8th Finals,3,Bayern Munich,2,1,0,1,7,1,...,10,10,1.00,ASSISTS,,Zlatan Ibrahimović,AC Milan,8,5,0.62
3,2011/12,8th Finals,4,Chelsea,2,1,0,1,5,4,...,10,7,0.70,ASSISTS,,Isaac Cuenca,FC Barcelona,8,5,0.62
4,2011/12,8th Finals,5,Milan AC,2,1,0,1,4,3,...,8,6,0.75,ASSISTS,,Kaká,Real Madrid,7,5,0.71


In [7]:
df.tail()

,YEAR,ROUND,RANK ER,TEAM ER,MP,W,D,L,GF,GA,...,MP SC,G,GOALS RATIO,CATEGORIES 3,RANK A,NAME A,TEAM A,MP A,A,ASSISTS RATIO
205,2017/18,Semi Finals,2,Liverpool FC,2,1,0,1,7,6,...,6,4,0.67,ASSISTS,,Radja Nainggolan,AS Roma,11,2,0.18
206,2017/18,Semi Finals,3,AS Roma,2,1,0,1,6,7,...,6,4,0.67,ASSISTS,,Lucas Vázquez,Real Madrid,10,2,0.20
207,2017/18,Semi Finals,4,Bayern Munich,2,0,1,1,3,4,...,11,3,0.27,ASSISTS,,Pablo Sarabia,Sevilla FC,10,2,0.20
208,2017/18,Finals,1,Real Madrid,1,1,0,0,3,1,...,10,3,0.30,ASSISTS,,Sergio Escudero,Sevilla FC,10,2,0.20
209,2017/18,Finals,2,Liverpool FC,1,0,0,1,1,3,...,10,3,0.30,ASSISTS,,Thomas Müller,Bayern Munich,10,2,0.20


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 42 columns):
YEAR             210 non-null object
ROUND            210 non-null object
RANK ER          210 non-null int64
TEAM ER          210 non-null object
MP               210 non-null int64
W                210 non-null int64
D                210 non-null int64
L                210 non-null int64
GF               210 non-null int64
GA               210 non-null int64
GD               210 non-null int64
Last 5           210 non-null object
CS%              210 non-null int64
BTTS%            210 non-null int64
FTS%             210 non-null int64
Over 1.5+ %      210 non-null int64
Over 2.5+ %      210 non-null int64
AVG              210 non-null float64
CATEGORIES 1     210 non-null object
RANK DS          210 non-null int64
TEAM DS          210 non-null object
GOALS            210 non-null int64
SHOTS            210 non-null float64
YELLOW CARD      210 non-null int64
RED CARD         210 

<i>Ok, we can notice that the dataset seems to be correct, the data are indeed organized from 2011 to 2019, which is the frame we intend to use for the prediction.

However, it's possible to highlight that there are few variables that are non-numerical (which is an issue for applying the dataset to the Machine Learning model, but we will correct it later), also the columns' names seem to be complicated to interpret for people who doesn't have a clue about football, so we need to arrange them in a simple way.</i>

#### Making extra data cleaning for optimization

In [9]:
### Re-arranging some of the columns' names so that they are easier to understand for everybody

df = df.rename(columns = {
    "YEAR" : "year",
    "ROUND" : "round",
    "RANK ER" : "team_rank",
    "TEAM ER" : "team",
    "MP" : "n_matchs_played",
    "W" : "n_wins",
    "D" : "n_draws",
    "L" : "n_loss",
    "GF" : "goals_scored",
    "GA" : "goals_conceded",
    "GD" : "goals_difference",
    "Last 5" : "last_results",
    "CS%" : "clean_sheets",
    "BTTS%" : "both_teams_to_score",
    "FTS%" : "fail_to_score",
    "Over 1.5+ %" : "over_1.5_goals",
    "Over 2.5+ %" : "over_2.5_goals",
    "AVG" : "average_goals",
    "CATEGORIES 1" : "first_category",
    "RANK DS" : "team_rank_detailed_stats",
    "TEAM DS" : "team_names_detailed_stats",
    "GOALS" : "total_goals",
    "SHOTS" : "shots_average",
    "YELLOW CARD" : "yellow_card",
    "RED CARD" : "red_card",
    "POSSESSION" : "ball_possession_percentage",
    "PASS%" : "pass_accuracy_percentage",
    "RATING" : "team_rating",
    "CATEGORIES 2" : "second_category",
    "RANK SC" : "scorer_rank",
    "NAME SC" : "scorer_name",
    "TEAM SC" : "scorer_team",
    "MP SC" : "scorer_match_played",
    "G" : "goals_numbers",
    "GOALS RATIO" : "goals_ratio",
    "CATEGORIES 3" : "third_category",
    "RANK A" : "assist_rank",
    "NAME A" : "assist_name",
    "TEAM A" : "assist_team",
    "MP A" : "assist_match_played",
    "A" : "assists_numbers",
    "ASSISTS RATIO" : "assists_ratio"
})

In [12]:
### Checking if the columns are correctly modified

df.columns

Index(['year', 'round', 'team_rank', 'team', 'n_matchs_played', 'n_wins',
       'n_draws', 'n_loss', 'goals_scored', 'goals_conceded',
       'goals_difference', 'last_results', 'clean_sheets',
       'both_teams_to_score', 'fail_to_score', 'over_1.5_goals',
       'over_2.5_goals', 'average_goals', 'first_category',
       'team_rank_detailed_stats', 'team_names_detailed_stats', 'total_goals',
       'shots_average', 'yellow_card', 'red_card',
       'ball_possession_percentage', 'pass_accuracy_percentage', 'team_rating',
       'second_category', 'scorer_rank', 'scorer_name', 'scorer_team',
       'scorer_match_played', 'goals_numbers', 'goals_ratio', 'third_category',
       'assist_rank', 'assist_name', 'assist_team', 'assist_match_played',
       'assists_numbers', 'assists_ratio'],
      dtype='object')

<i>Ok, looks like all of the features' names have been correctly changed !</i>

In [13]:
### Checking if there are type issues for each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 42 columns):
year                          210 non-null object
round                         210 non-null object
team_rank                     210 non-null int64
team                          210 non-null object
n_matchs_played               210 non-null int64
n_wins                        210 non-null int64
n_draws                       210 non-null int64
n_loss                        210 non-null int64
goals_scored                  210 non-null int64
goals_conceded                210 non-null int64
goals_difference              210 non-null int64
last_results                  210 non-null object
clean_sheets                  210 non-null int64
both_teams_to_score           210 non-null int64
fail_to_score                 210 non-null int64
over_1.5_goals                210 non-null int64
over_2.5_goals                210 non-null int64
average_goals                 210 non-null float64
first_c

<i>We can observe here that all the types seem to be coherent for each features contents, however, we'll have to convert all of the categorical features to numerical values so that they will be able to be processed by the Machine Learning models, but this will be done later.</i>

In [15]:
### Checking if there are missing values

df.isnull().sum()

year                          0
round                         0
team_rank                     0
team                          0
n_matchs_played               0
n_wins                        0
n_draws                       0
n_loss                        0
goals_scored                  0
goals_conceded                0
goals_difference              0
last_results                  0
clean_sheets                  0
both_teams_to_score           0
fail_to_score                 0
over_1.5_goals                0
over_2.5_goals                0
average_goals                 0
first_category                0
team_rank_detailed_stats      0
team_names_detailed_stats     0
total_goals                   0
shots_average                 0
yellow_card                   0
red_card                      0
ball_possession_percentage    0
pass_accuracy_percentage      0
team_rating                   0
second_category               0
scorer_rank                   7
scorer_name                   0
scorer_t

<i>We can notice there are missing values in the <b>scorer_rank</b> and in the <b>assist_rank</b> (there are missing values but for no reasons they aren't displayed above) columns.

Actually, this issue can be explained by the fact that while ranking the scorers and the assists players, it often happen that a lot of played have scored or made the same amounts of goals/assists, so statisticians just put the rank number one time for all of these players.

For example, if you have five players that have scored 5 goals in the whole competitions, they'll be ranked like this :</i>

<b>

Rank  Player  Goals

4    Player_1      5 <br>
&nbsp;&nbsp;       Player_2      5 <br>
&nbsp;&nbsp;      Player_3      5 <br>
&nbsp;&nbsp;      Player_4      5 <br>
&nbsp;&nbsp;      Player_5      5 <br>

</b>

<i>To deal with those missing values, as we have a small dataset in terms of observations, we can manually fill those missing values.<br> Usually, with a 'normal' dataset that have more than 1000 rows, we would have replaced the missing values by imputing them with the mean or the median value of the distribution.</i>

In [16]:
### Checking if it's worth replacing the missing values or if we should just simply drop the column with these values
### To do this, we calculate the percentage of missing values for each feature

na_percentage = df.isnull().mean() * 100

na_percentage

year                          0.000000
round                         0.000000
team_rank                     0.000000
team                          0.000000
n_matchs_played               0.000000
n_wins                        0.000000
n_draws                       0.000000
n_loss                        0.000000
goals_scored                  0.000000
goals_conceded                0.000000
goals_difference              0.000000
last_results                  0.000000
clean_sheets                  0.000000
both_teams_to_score           0.000000
fail_to_score                 0.000000
over_1.5_goals                0.000000
over_2.5_goals                0.000000
average_goals                 0.000000
first_category                0.000000
team_rank_detailed_stats      0.000000
team_names_detailed_stats     0.000000
total_goals                   0.000000
shots_average                 0.000000
yellow_card                   0.000000
red_card                      0.000000
ball_possession_percentag

<i>Nice ! It seems that the only column that got potential missing values is only about 3,3% of the entire value frame.<br>
Consequently, we can in fact manually replace them.</i>

In [152]:
### Function to fill a column content at a specific row

def fill_column(column_name, min_val, max_val, value_input):
    df[column_name].iloc[min_val:max_val] = value_input

In [153]:
### Filling the missing values for the scorer_rank column

fill_column('scorer_rank', 6, 13, 6)
fill_column('scorer_rank', 14, 20, 14)
fill_column('scorer_rank', 21, 30, 21)
fill_column('scorer_rank', 33, 35, 3)
fill_column('scorer_rank', 36, 40, 6)
fill_column('scorer_rank', 41, 50, 11)
fill_column('scorer_rank', 51, 60, 21)
fill_column('scorer_rank', 63, 64, 3)
fill_column('scorer_rank', 65, 67, 5)
fill_column('scorer_rank', 68, 72, 8)
fill_column('scorer_rank', 73, 81, 13)
fill_column('scorer_rank', 82, 90, 22)
fill_column('scorer_rank', 91, 93, 2)
fill_column('scorer_rank', 95, 98, 5)
fill_column('scorer_rank', 99, 102, 9)
fill_column('scorer_rank', 103, 106, 13)
fill_column('scorer_rank', 107, 109, 17)
fill_column('scorer_rank', 110, 120, 20)
fill_column('scorer_rank', 123, 124, 3)
fill_column('scorer_rank', 126, 127, 6)
fill_column('scorer_rank', 128, 131, 8)
fill_column('scorer_rank', 132, 135, 12)
fill_column('scorer_rank', 136, 150, 16)
fill_column('scorer_rank', 153, 154, 3)
fill_column('scorer_rank', 156, 157, 6)
fill_column('scorer_rank', 158, 162, 8)
fill_column('scorer_rank', 163, 172, 13)
fill_column('scorer_rank', 173, 180, 23)
fill_column('scorer_rank', 182, 184, 2)
fill_column('scorer_rank', 185, 186, 5)
fill_column('scorer_rank', 187, 188, 7)
fill_column('scorer_rank', 189, 190, 9)
fill_column('scorer_rank', 191, 196, 11)
fill_column('scorer_rank', 197, 207, 17)
fill_column('scorer_rank', 208, 210, 28)

In [207]:
### Filling the missing values for the assist_rank column

fill_column('assist_rank', 2, 6, 2)
fill_column('assist_rank', 7, 14, 7)
fill_column('assist_rank', 15, 21, 15)
fill_column('assist_rank', 22, 30, 2)
fill_column('assist_rank', 33, 41, 3)
fill_column('assist_rank', 42, 53, 12)
fill_column('assist_rank', 54, 60, 24)
fill_column('assist_rank', 61, 63, 1)
fill_column('assist_rank', 64, 72, 4)
fill_column('assist_rank', 73, 81, 13)
fill_column('assist_rank', 82, 90, 22)
fill_column('assist_rank', 92, 95, 2)
fill_column('assist_rank', 96, 105, 6)
fill_column('assist_rank', 106, 120, 16)
fill_column('assist_rank', 121, 122, 1)
fill_column('assist_rank', 123, 127, 3)
fill_column('assist_rank', 128, 137, 8)
fill_column('assist_rank', 138, 150, 18)
fill_column('assist_rank', 153, 154, 3)
fill_column('assist_rank', 155, 156, 5)
fill_column('assist_rank', 157, 167, 7)
fill_column('assist_rank', 168, 180, 18)
fill_column('assist_rank', 183, 186, 3)
fill_column('assist_rank', 187, 201, 7)
fill_column('assist_rank', 202, 210, 22)

In [209]:
df.iloc[162:,29:35]

,scorer_rank,scorer_name,scorer_team,scorer_match_played,goals_numbers,goals_ratio
162,13,Saúl Ñíguez,Atletico Madrid,12,4,0.33
163,13,Paulo Dybala,Juventus FC,11,4,0.36
164,13,Riyad Mahrez,Leicester City,9,4,0.44
165,13,Neymar,FC Barcelona,9,4,0.44
166,13,Mesut Özil,Arsenal,8,4,0.50
167,13,André Silva,FC Porto,8,4,0.50
168,13,Ángel Di María,Paris Saint-Germain,7,4,0.57
169,13,Theo Walcott,Arsenal,6,4,0.67
170,13,Arda Turan,FC Barcelona,5,4,0.80
171,13,Marco Reus,Borussia Dortmund,4,4,1.00


In [210]:
df.iloc[156:,36:]

,assist_rank,assist_name,assist_team,assist_match_played,assists_numbers,assists_ratio
156,7,Miralem Pjanic,Juventus FC,12,3,0.25
157,7,Fabinho,AS Monaco,10,3,0.30
158,7,Thiago,Bayern Munich,9,3,0.33
159,7,Douglas Costa,Bayern Munich,9,3,0.33
160,7,Faouzi Ghoulam,SSC Napoli,8,3,0.38
161,7,Alexis Sánchez,Arsenal,8,3,0.38
162,7,Mesut Özil,Arsenal,8,3,0.38
163,7,Benjamin Mendy,AS Monaco,7,3,0.43
164,7,Raheem Sterling,Manchester City,7,3,0.43
165,7,Ludwig Augustinsson,FC Copenhagen,6,3,0.50


In [211]:
### Double checking if all of the missing values are gone

df.isnull().sum()

year                          0
round                         0
team_rank                     0
team                          0
n_matchs_played               0
n_wins                        0
n_draws                       0
n_loss                        0
goals_scored                  0
goals_conceded                0
goals_difference              0
last_results                  0
clean_sheets                  0
both_teams_to_score           0
fail_to_score                 0
over_1.5_goals                0
over_2.5_goals                0
average_goals                 0
first_category                0
team_rank_detailed_stats      0
team_names_detailed_stats     0
total_goals                   0
shots_average                 0
yellow_card                   0
red_card                      0
ball_possession_percentage    0
pass_accuracy_percentage      0
team_rating                   0
second_category               0
scorer_rank                   0
scorer_name                   0
scorer_t

In [212]:
### Overviewing the data one more time

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 42 columns):
year                          210 non-null object
round                         210 non-null object
team_rank                     210 non-null int64
team                          210 non-null object
n_matchs_played               210 non-null int64
n_wins                        210 non-null int64
n_draws                       210 non-null int64
n_loss                        210 non-null int64
goals_scored                  210 non-null int64
goals_conceded                210 non-null int64
goals_difference              210 non-null int64
last_results                  210 non-null object
clean_sheets                  210 non-null int64
both_teams_to_score           210 non-null int64
fail_to_score                 210 non-null int64
over_1.5_goals                210 non-null int64
over_2.5_goals                210 non-null int64
average_goals                 210 non-null float64
first_c

<i>After double checking the dataset, we can affirm that there are no more missing values and each feature seems to have an appropriate type. <br><br>
Now, before doing the analysis, for more visibility and practicity, we are going to separate the entire dataset into 4 subsets, so at the end we will have :
    - one complete dataset with the teams' results per round, the teams' detailed stats in the whole competition and the players' stats (called "df")
    - one subset that only contains the teams' results per round (called "teams_round")
    - one subset that only contains the teams' detailed stats (called "teams_detailed_stats")
    - one subset that only contains the best scorers stats (called "player_scorers")
    - one subset that only contains the best assists players stats (called "player_assists")

Indeed, to achieve our winner prediction, we are going to predict :
    - the best team in each round of the competition
    - the best team in terms of detailed stats
    - the best scorer of the competition
    - the best assist player of the competition
    
Then, by with these three predictions, we'll be able to make hypothesis on the team that is most likely winning the competition based on the probabilities predicted.

Moreover, those predictions will also allow us to answer these questions :
    - Does an offensive style make a team win the tournament ?
    - Does an defensive style make a team win the tournament ?
    - Does an hybrid style make a team win the tournament ?
    - Does the best scorer of the tournament increases the teams' chances of winning the tournament ?
    - Does the best assist player of the tournament increases the teams' chances of winning the tournament ?
    - What are the chances of a team that has bad stats during specific round of the tournament of winning the tournament ?
    - Is a team 100% sure of winning the tournament if he is ranked first in the detailed stats ? (which means that he globally performed well in the tournament, but sometimes, upset can happens during the elimination phase against "surprise" teams !)
    - Does a team need to perform incredibly well in each round of the competition in order to win the competition ? (sometimes, it happens that a team that is very average has a VERY good game one time in one round against a team that is supposed better then them, which is kind of considered as a "Hold-Up")
</i>

#### Splitting the dataset into subsets

In [213]:
### The complete dataset overview

df.head()

,year,round,team_rank,team,n_matchs_played,n_wins,n_draws,n_loss,goals_scored,goals_conceded,...,scorer_match_played,goals_numbers,goals_ratio,third_category,assist_rank,assist_name,assist_team,assist_match_played,assists_numbers,assists_ratio
0,2011/12,8th Finals,1,FC Barcelona,2,2,0,0,10,2,...,11,14,1.27,ASSISTS,1,Lionel Messi,FC Barcelona,11,9,0.82
1,2011/12,8th Finals,2,Real Madrid,2,1,1,0,5,2,...,11,12,1.09,ASSISTS,2,Karim Benzema,Real Madrid,10,5,0.50
2,2011/12,8th Finals,3,Bayern Munich,2,1,0,1,7,1,...,10,10,1.00,ASSISTS,2,Zlatan Ibrahimović,AC Milan,8,5,0.62
3,2011/12,8th Finals,4,Chelsea,2,1,0,1,5,4,...,10,7,0.70,ASSISTS,2,Isaac Cuenca,FC Barcelona,8,5,0.62
4,2011/12,8th Finals,5,Milan AC,2,1,0,1,4,3,...,8,6,0.75,ASSISTS,2,Kaká,Real Madrid,7,5,0.71


In [222]:
### Features' overview in order to facilitate the selection for the subsets

df.columns

Index(['year', 'round', 'team_rank', 'team', 'n_matchs_played', 'n_wins',
       'n_draws', 'n_loss', 'goals_scored', 'goals_conceded',
       'goals_difference', 'last_results', 'clean_sheets',
       'both_teams_to_score', 'fail_to_score', 'over_1.5_goals',
       'over_2.5_goals', 'average_goals', 'first_category',
       'team_rank_detailed_stats', 'team_names_detailed_stats', 'total_goals',
       'shots_average', 'yellow_card', 'red_card',
       'ball_possession_percentage', 'pass_accuracy_percentage', 'team_rating',
       'second_category', 'scorer_rank', 'scorer_name', 'scorer_team',
       'scorer_match_played', 'goals_numbers', 'goals_ratio', 'third_category',
       'assist_rank', 'assist_name', 'assist_team', 'assist_match_played',
       'assists_numbers', 'assists_ratio'],
      dtype='object')

In [218]:
### Creating the first subset "teams_round"

teams_round = df.loc[:, 'year':'average_goals']

teams_round.head()

,year,round,team_rank,team,n_matchs_played,n_wins,n_draws,n_loss,goals_scored,goals_conceded,goals_difference,last_results,clean_sheets,both_teams_to_score,fail_to_score,over_1.5_goals,over_2.5_goals,average_goals
0,2011/12,8th Finals,1,FC Barcelona,2,2,0,0,10,2,8,WW,42,50,17,83,75,3.75
1,2011/12,8th Finals,2,Real Madrid,2,1,1,0,5,2,3,DW,50,50,0,92,75,3.66
2,2011/12,8th Finals,3,Bayern Munich,2,1,0,1,7,1,6,LW,38,46,15,92,38,2.85
3,2011/12,8th Finals,4,Chelsea,2,1,0,1,5,4,1,LW,38,62,0,83,62,3.47
4,2011/12,8th Finals,5,Milan AC,2,1,0,1,4,3,1,WL,40,50,20,80,60,3.00


In [220]:
### Creating the second subset "teams_detailed_stats"

col_1 = ['year', 'team_rank_detailed_stats', 'team_names_detailed_stats', 'total_goals', 'shots_average', 'yellow_card', 'red_card',
              'ball_possession_percentage', 'pass_accuracy_percentage', 'team_rating']

teams_detailed_stats = df[col_1]

teams_detailed_stats.head()

,year,team_rank_detailed_stats,team_names_detailed_stats,total_goals,shots_average,yellow_card,red_card,ball_possession_percentage,pass_accuracy_percentage,team_rating
0,2011/12,1,Chelsea,25,14.8,31,1,47.6,82.5,7.19
1,2011/12,2,Real Madrid,35,19.1,22,1,56.1,85.9,7.15
2,2011/12,3,FC Barcelona,35,19.5,17,0,68.2,90.5,7.15
3,2011/12,4,Bayern Munich,26,19.1,27,1,55.1,86.2,7.11
4,2011/12,5,Valencia CF,12,16.7,11,0,58.5,85.1,7.04


In [223]:
### Creating the third subset "scorers"

col_2 = ['year', 'scorer_rank', 'scorer_name', 'scorer_team', 'scorer_match_played', 'goals_numbers', 'goals_ratio']

scorers = df[col_2]

scorers.head()

,year,scorer_rank,scorer_name,scorer_team,scorer_match_played,goals_numbers,goals_ratio
0,2011/12,1,Lionel Messi,FC Barcelona,11,14,1.27
1,2011/12,2,Mario Gomez,Bayern Munich,11,12,1.09
2,2011/12,3,Cristiano Ronaldo,Real Madrid,10,10,1.00
3,2011/12,4,Karim Benzema,Real Madrid,10,7,0.70
4,2011/12,5,Didier Drogba,Chelsea,8,6,0.75


In [224]:
### Creating the fourth subset "assists"

col_3 = ['year', 'assist_rank', 'assist_name', 'assist_team', 'assist_match_played', 'assists_numbers', 'assists_ratio']

assists = df[col_3]

assists.head()

,year,assist_rank,assist_name,assist_team,assist_match_played,assists_numbers,assists_ratio
0,2011/12,1,Lionel Messi,FC Barcelona,11,9,0.82
1,2011/12,2,Karim Benzema,Real Madrid,10,5,0.50
2,2011/12,2,Zlatan Ibrahimović,AC Milan,8,5,0.62
3,2011/12,2,Isaac Cuenca,FC Barcelona,8,5,0.62
4,2011/12,2,Kaká,Real Madrid,7,5,0.71


#### Descriptive analysis of the data

In [229]:
### Checking class imbalance for each subset target

## For the teams results in each round of the competition

print('Total number of observations : {}'.format(teams_round.shape[0]))
teams_round.team.value_counts()

Total number of observations : 210


Real Madrid             25
Bayern Munich           22
FC Barcelona            18
Juventus FC             13
Atlético Madrid         13
Chelsea                 10
Paris Saint-Germain     10
Borussia Dortmund        9
Manchester City          8
Arsenal                  6
FC Porto                 5
AS Monaco                5
SL Benfica               5
AS Roma                  4
Bayer Leverkusen         4
Liverpool FC             4
Manchester United        4
Sevilla FC               3
Shakhtar Donetsk         3
FC Basel                 3
Schalke 04               3
Galatasaray              3
Malaga                   2
AC Milan                 2
APOEL Nicosie            2
SSC Napoli               2
Leicester City           2
Zenit St Petersburg      2
Marseille                2
VfL Wolfsburg            2
Milan AC                 2
Tottenham Hotspur FC     1
KAA Gent                 1
Besiktas                 1
Valencia CF              1
Lyon                     1
PSV Eindhoven            1
C

In [230]:
## For the teams rankings depending on the detailed stats during the whole competition

print('Total number of observations : {}'.format(teams_detailed_stats.shape[0]))
teams_detailed_stats.team_names_detailed_stats.value_counts()

Total number of observations : 210


Real Madrid               7
FC Barcelona              7
SL Benfica                7
Manchester City           7
FC Porto                  7
Bayern Munich             7
Juventus FC               6
Chelsea                   6
Shakhtar Donetsk          6
Arsenal                   6
CSKA Moscow               6
Olympiakos                6
Paris Saint-Germain       6
Borussia Dortmund         6
Bayer Leverkusen          5
Manchester United         5
Atletico Madrid           5
Zenit St Petersburg       5
FC Basel                  5
Galatasaray               4
SSC Napoli                4
Ajax Amsterdam            4
Anderlecht                3
AC Milan                  3
Celtic                    3
Dynamo Kyiv               3
Valencia CF               3
Schalke 04                3
Lyon                      3
Sporting CP               3
                         ..
Spartak Moscow            2
Maribor                   2
Marseille                 2
PSV Eindhoven             2
Liverpool FC        

In [231]:
### For the scorers

print('Total number of observations : {}'.format(scorers.shape[0]))
scorers.scorer_name.value_counts()

Total number of observations : 210


Cristiano Ronaldo            7
Lionel Messi                 7
Karim Benzema                7
Thomas Müller                6
Robert Lewandowski           6
Mario Mandzukic              5
Edinson Cavani               5
Neymar                       4
Sergio Agüero                4
Gonzalo Higuaín              4
Zlatan Ibrahimovic           3
Luis Suárez                  3
Marco Reus                   3
Arjen Robben                 3
Pierre-Emerick Aubameyang    3
Hulk                         3
Seydou Doumbia               3
Kylian Mbappé                2
Arturo Vidal                 2
Arda Turan                   2
Klaas-Jan Huntelaar          2
Jackson Martínez             2
Antoine Griezmann            2
Álvaro Morata                2
Alex Teixeira                2
Luiz Adriano                 2
Roberto Soldado              2
Fernando Torres              2
Alexis Sánchez               2
Julian Draxler               2
                            ..
Harry Kane                   1
Yacine B

In [233]:
### For the assists players

print('Total number of observations : {}'.format(assists.shape[0]))
assists.assist_name.value_counts()

Total number of observations : 210


Cristiano Ronaldo            6
Luis Suárez                  4
Thomas Müller                4
Neymar                       4
Lionel Messi                 4
Dani Alves                   4
Karim Benzema                3
Toni Kroos                   3
Juanfran                     3
Ángel Di María               3
Ezequiel Lavezzi             3
Mesut Özil                   3
Marcelo                      3
Isco                         3
Philipp Lahm                 2
David Alaba                  2
Cesc Fàbregas                2
Hakan Calhanoglu             2
Alexis Sánchez               2
Dani Carvajal                2
Bastian Schweinsteiger       2
Wayne Rooney                 2
Sami Khedira                 2
Franck Ribéry                2
Nicolás Gaitán               2
Douglas Costa                2
Andrés Iniesta               2
Gregory van der Wiel         2
James Milner                 2
James Rodríguez              2
                            ..
Mario Balotelli              1
Alan Dza

##### First highlights

<i>
With this first quick analysis, we can already notice that there are four teams (also called the Big Four in Europe) that are very dominant in this competitions :
    - Real Madrid (a 13 times Champions League winner, one of the two best Spanish team with the FC Barcelona)
    - Bayern Munich (a 5 times Champions League winner, the best German team since almost 8 years in a row)
    - FC Barcelona (a 5 times Champions League winner, one of the two superpower in Spain with the Real Madrid)
    - Juventus FC (a 1 time Champions League winner, the best Italian team since 9 years in a row)
    
Indeed, by calculating the occurencies of each team we see that :
    - Real Madrid represents 11% of the observations
    - Bayern Munich represents 10,4% of the observations
    - FC Barcelona represents 8,6% of the observations
    - Juventus FC represents 6,2% of the observations

Those four teams already represent 36,2% of the total observations of the dataset.

Furthermore, regarding the players analysis, we can notice that the players that have the best frequencies are :
    - Cristiano Ronaldo (played for Real Madrid from 2009 to 2018 and is currently playing at Juventus FC in 2019, the best scorer in UCL since 2013)
    - Lionel Messi (playing for FC Barcelona, the second best scorer in the history of the UCL behind Cristiano Ronaldo and a genius assist player)
    - Robert Lewandowski (playing for Bayern Munich, the best scorer in Germany since 2012)
    - Mario Mandzukic & Gonzalo Higuain (playing for Juventus FC, among the best scorers in the world)
    
So, even for the players, it seems that the best ones are those with the best frequencies, which quite logical if their respective teams are performing well.

We may have a first correlation between players and teams performances and an answer to one of our questions above here !
</i>